# Data EDA

* 데이터 라벨

|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|생활문화|스포츠|사회|세계|IT과학|경제|정치|

# Library import 단계

In [1]:
import transformers
import torch
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from tqdm import tqdm

tqdm.pandas()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Noise vs Non - Noise 분류
- Ko-Llama3 사용

In [2]:
# train_data 불러오기
data = pd.read_csv('./data/train.csv')
data.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


- 초기 실행 시 10m 13.4s 소요(m2max, ram 32GB 기준). 
- 재실행시 37~8s
- 모델 용량 4.89 + 4.92 + 5.00 + 1.55GB

In [3]:
model_id = "saltlux/Ko-Llama3-Luxia-8B"

pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto",
    max_new_tokens=50,
)
pipeline("<|begin_of_text|>안녕하세요. 솔트룩스 AI Labs 입니다.")

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.23s/it]


[{'generated_text': "<|begin_of_text|>안녕하세요. 솔트룩스 AI Labs 입니다. 이번 포스팅에서는 솔트룩스 AI Labs에서 진행한 '2020년 1분기 AI 기술 트렌드'를 소개해 드리려고 합니다. 2020년 1분기 AI 기술 트렌드에서는 2020"}]

In [4]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [12]:
prompt_template = """
다음은 한국어 뉴스 제목입니다:
"{text}"
이 제목이 노이즈가 포함되어 있다면 'Yes', 그렇지 않다면 'No'로 답변하세요. 오직 'Yes' 혹은 'No' 한 단어로만 답변하세요.

### 예시 1
text: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보, 당신의 답변: Yes

### 예시 2
text: 갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩, 당신의 답변: No

### 예시 3
text: 게시판 KISA 박민정 책임연구원 APTLD 이사 선출, 당신의 답변: No
"""

Ko-Llama3 사용 텍스트 노이즈 감지
- 소요 시간 2 ~ 3 시간. 1 iter 당 3.4 ~ 3.5 소요

In [13]:
def detect_noise(df: pd.DataFrame) -> pd.DataFrame:
    descriptions = []
    for text in tqdm(df['text'], desc="Processing"):
        prompt = prompt_template.format(text=text)
        response = llm.invoke(prompt)
        print(f"text: {text}, noise: {response}")
        if 'Yes' in response:
            descriptions.append('Yes')
        elif 'No' in response:
            descriptions.append('No')
        else:
            descriptions.append('Uncertain')       
    df['description'] = descriptions
    return df


noise_detected_data = detect_noise(data)
noise_detected_data

Processing:   0%|          | 1/2800 [00:02<1:40:27,  2.15s/it]

text: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보, noise: 
다음은 한국어 뉴스 제목입니다:
"정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보"
이 제목이 노이즈가 포함되어 있다면 'Yes', 그렇지 않다면 'No'로 답변하세요. 오직 'Yes' 혹은 'No' 한 단어로만 답변하세요.

### 예시 1
text: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보, 당신의 답변: Yes

### 예시 2
text: 갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩, 당신의 답변: No

### 예시 3
text: 게시판 KISA 박민정 책임연구원 APTLD 이사 선출, 당신의 답변: No
### 예시 4
text: 2017년 1월 1일부터 2017년 1월 31일까지의 1개월간의 데이터를 분석한 결과, 1월 1일부터 1


Processing:   0%|          | 2/2800 [00:04<1:39:36,  2.14s/it]

text: K찰.국DLwo 로L3한N% 회장 2 T0&}송=, noise: 
다음은 한국어 뉴스 제목입니다:
"K찰.국DLwo 로L3한N% 회장 2 T0&}송="
이 제목이 노이즈가 포함되어 있다면 'Yes', 그렇지 않다면 'No'로 답변하세요. 오직 'Yes' 혹은 'No' 한 단어로만 답변하세요.

### 예시 1
text: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보, 당신의 답변: Yes

### 예시 2
text: 갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩, 당신의 답변: No

### 예시 3
text: 게시판 KISA 박민정 책임연구원 APTLD 이사 선출, 당신의 답변: No
### 예시 4
text: K찰.국DLwo 로L3한N% 회장 2 T0&}송=, 당신의 답변: Yes

### 예시 5
text: 1.5조원 규모 '


Processing:   0%|          | 3/2800 [00:06<1:39:26,  2.13s/it]

...

text: 아이`XSI수리0* b대`…맥3 디dF레< 41/', noise: 
다음은 한국어 뉴스 제목입니다:
"아이`XSI수리0* b대`…맥3 디dF레< 41/'"
이 제목이 노이즈가 포함되어 있다면 'Yes', 그렇지 않다면 'No'로 답변하세요. 오직 'Yes' 혹은 'No' 한 단어로만 답변하세요.

### 예시 1
text: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보, 당신의 답변: Yes

### 예시 2
text: 갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩, 당신의 답변: No

### 예시 3
text: 게시판 KISA 박민정 책임연구원 APTLD 이사 선출, 당신의 답변: No
### 예시 4
text: 1분기 1.7% 성장…2분기 2.9% 전망, 당신의 답변: Yes
### 예시 5
text: 1분기 1.7%


Processing:   1%|          | 15/2800 [00:31<1:38:50,  2.13s/it]

text: 문/인 당2 4nS 민관2동7사위 /"X보 철거tt, noise: 
다음은 한국어 뉴스 제목입니다:
"문/인 당2 4nS 민관2동7사위 /"X보 철거tt"
이 제목이 노이즈가 포함되어 있다면 'Yes', 그렇지 않다면 'No'로 답변하세요. 오직 'Yes' 혹은 'No' 한 단어로만 답변하세요.

### 예시 1
text: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보, 당신의 답변: Yes

### 예시 2
text: 갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩, 당신의 답변: No

### 예시 3
text: 게시판 KISA 박민정 책임연구원 APTLD 이사 선출, 당신의 답변: No
### 예시 4
text: 2017년 1월 1일부터 2017년 12월 31일까지의 기간 동안, 당신의 답변: Yes

### 예시 5
text: 2017년


Processing:   1%|          | 15/2800 [00:33<1:43:20,  2.23s/it]


KeyboardInterrupt: 

In [8]:
noise_data_count = len(data[data['description'] == 'Yes'])
noise_data_count

2800